In [1]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from evaluate import load
import pandas as pd
import numpy as np
from datasets import load_dataset
import torch
import json

c:\Users\carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "deepset/electra-base-squad2"

In [9]:
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

In [3]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading: 100%|██████████| 635/635 [00:00<00:00, 635kB/s]
c:\Users\carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\carlos\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 436M/436M [01:00<00:00, 7.

In [4]:
model.eval()    

ElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768

In [5]:
raw_datasets = load_dataset("squad_v2")

print("Context: ", raw_datasets["validation"][0]["context"])
print("Question: ", raw_datasets["validation"][0]["question"])
print("Answer: ", raw_datasets["validation"][0]["answers"])

Found cached dataset squad_v2 (C:/Users/carlos/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
100%|██████████| 2/2 [00:00<00:00, 12.15it/s]

Context:  The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
Question:  In what country is Normandy located?
Answer:  {'text': ['France', 'France', 'France', 'France'], 'answer_start': [159, 159, 159, 159]}


In [6]:
raw_datasets["validation"].filter(lambda x: len(x["answers"]["text"]) != 1)
print(raw_datasets["validation"][0]["answers"])
print(raw_datasets["validation"][2]["answers"])

Loading cached processed dataset at C:\Users\carlos\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d\cache-a729e31eeecedd01.arrow


{'text': ['France', 'France', 'France', 'France'], 'answer_start': [159, 159, 159, 159]}
{'text': ['Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway'], 'answer_start': [256, 256, 256, 256]}


In [7]:
def get_predictions(question, context):
    QA_input = {
        'question': question,
        'context': context
    }
    return nlp(QA_input)

In [10]:
validation_ds = raw_datasets["validation"]
predictions = {}
for item in validation_ds:
    question = item["question"]
    context = item["context"]
    prediction = get_predictions(question, context)
    predictions[item["id"]] = prediction

with open("predictions.json", "w") as outfile:
    json.dump(predictions, outfile)